Since `darkwing` is based on the DuckDB relational API, all expressions are "lazy" in that they defer evaluation until a result or a data preview is requested.
This allows us to build up complex data processing pipelines iteratively, but without needing to compute extranous intermediate results. Instead, under the hood, DuckDB will gather the sequence of steps and pass it to a query optimizer, which will apply optimizations like predicate and projection pushdown. The full operation will be executed by DuckDB making full use of all the cores available on your machine, streaming the operations if possible, and even spilling to disk if the operation is too large to fit in memory.

In [1]:
import darkwing as dw

In [2]:
t0 = dw.Table('data/yellow_tripdata_2010-01.parquet')
t0.columns

['vendor_id',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'rate_code',
 'store_and_fwd_flag',
 'dropoff_longitude',
 'dropoff_latitude',
 'payment_type',
 'fare_amount',
 'surcharge',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'total_amount']

In [3]:
count_rows = "select format('{:.2e}', 1.0*count(*)) as num_rows"
t0.do(count_rows)

┌──────────┐
│ num_rows │
│ varchar  │
├──────────┤
│ 1.49e+07 │
└──────────┘

The following code is executed instantaneously, since no query operations are performed.

In [4]:
t1 = t0.do(
    'where (pickup_longitude != 0) and (pickup_latitude != 0)',
    'where total_amount > 0',
    
    'select *, h3_latlng_to_cell(pickup_latitude, pickup_longitude, 12) as hexid',
    'select * replace ( h3_h3_to_string(hexid) as hexid )',
    'select cast(pickup_datetime as timestamp) as ts, hexid, total_amount as amt',
)

Listing the table as the last expression in a Jupyter notebook makes Jupyter try to represent the table, which triggers DuckDB do either compute the full table, or, in the case that the table has many rows, compute just enough rows to show a preview. In many instances, the preview is faster to compute.

The following is still fast, but just a bit slower than the previous cell, since this is where the query associated with the operations above is actually performed.

In [5]:
t1

┌─────────────────────┬─────────────────┬───────────────────┐
│         ts          │      hexid      │        amt        │
│      timestamp      │     varchar     │      double       │
├─────────────────────┼─────────────────┼───────────────────┤
│ 2010-01-26 07:41:00 │ 8c2a100d45b01ff │               5.0 │
│ 2010-01-30 23:31:00 │ 8c2a107258e61ff │              16.3 │
│ 2010-01-18 20:22:20 │ 8c2a1008b82b5ff │              12.7 │
│ 2010-01-09 01:18:00 │ 8c2a100d65653ff │              14.3 │
│ 2010-01-18 19:10:14 │ 8c2a100d22945ff │              6.67 │
│ 2010-01-17 09:18:00 │ 8c2a10725ac5bff │               6.6 │
│ 2010-01-09 13:49:00 │ 8c2a100d620b7ff │               7.4 │
│ 2010-01-09 00:25:00 │ 8c2a1072c86abff │              12.3 │
│ 2010-01-27 18:15:00 │ 8c2a100d2bb69ff │              12.0 │
│ 2010-01-08 16:05:00 │ 8c2a107250403ff │              10.2 │
│          ·          │        ·        │                ·  │
│          ·          │        ·        │                ·  │
│       

In [6]:
t2 = t1.alias('tbl1').do("""
select
      a.hexid
    , a.ts as ts1
    , b.ts as ts2
    , a.amt as amt1
    , b.amt as amt2
from
    tbl1 as a
inner join
    tbl1 as b
using
    (hexid)
""")

Even though the computation for `t2` is complex, we can compute a preview fairly quickly. The following runs in about 2 seconds on my laptop.

In [7]:
t2

┌─────────────────┬─────────────────────┬─────────────────────┬───────────────────┬───────────────────┐
│      hexid      │         ts1         │         ts2         │       amt1        │       amt2        │
│     varchar     │      timestamp      │      timestamp      │      double       │      double       │
├─────────────────┼─────────────────────┼─────────────────────┼───────────────────┼───────────────────┤
│ 8c2a1008b0231ff │ 2010-01-02 13:28:13 │ 2010-01-20 19:18:00 │               7.2 │               4.8 │
│ 8c2a100d6c999ff │ 2010-01-02 11:48:34 │ 2010-01-20 23:45:41 │               5.4 │               7.5 │
│ 8c2a100888e3dff │ 2010-01-10 14:49:00 │ 2010-01-21 20:44:00 │               5.4 │               5.9 │
│ 8c2a1008bb9b1ff │ 2010-01-08 14:46:12 │ 2010-01-08 07:46:27 │              19.4 │               7.8 │
│ 8c2a103b0374dff │ 2010-01-08 16:03:15 │ 2010-01-14 17:30:53 │              51.0 │              28.4 │
│ 8c2a100892491ff │ 2010-01-06 18:56:38 │ 2010-01-26 12:15:00 │ 

However, running `count_rows` on `t2` forces the full join operation to be performed (previously, we only computed a *partial* join to dispaly the preview). The following takes about 50 seconds on my laptop.

Note that the row count for this intermediate table is about **10 billion rows**. We deal with the table directly here for demonstration purposes, but we as we continue the pipeline below, we will avoid ever forming this intermediate table.

In [8]:
# renders slowly because you have to do the full join
t2.do(count_rows)

┌──────────┐
│ num_rows │
│ varchar  │
├──────────┤
│ 1.05e+10 │
└──────────┘

Again, it is intantaneous to form the expression representing `t3`, as long as we don't need to compute the expression just yet.

Note that the timestamp filtering below could have also been given above as part of the join. We're free to do it either way and the performance will be identical because DuckDB will push the filters down in its query planning/optimization step.

In [9]:
t3 = t2.do(
    'where ts1 < ts2',
    'where ts2 < ts1 + interval 1 minute',
    'select hexid, max(abs(amt1-amt2)) as diff group by 1',
    'where diff > 0'
    'order by diff',
)

Note that this is faster than the `t2.do(count_rows)`, even though it does more work! This cell runs in about 44 seconds on my laptop.

This final result has about **29 thousand rows**, something much more reasonable to materialize directly as a Pandas dataframe, for instance.

In [10]:
t3.do(count_rows)

┌──────────┐
│ num_rows │
│ varchar  │
├──────────┤
│ 2.86e+04 │
└──────────┘

Get a Pandas dataframe in about 53 seconds.

In [11]:
t3.df()

,hexid,diff
0,8c2a100d2d12bff,0.01
1,8c2a100d2a94dff,0.02
2,8c2a1072c846bff,0.02
3,8c2a107253359ff,0.02
4,8c2a100891611ff,0.02
...,...,...
28563,8c2a10aa2cb13ff,175.88
28564,8c2a100f52815ff,180.45
28565,8c2a108f664e7ff,203.00
28566,8c2a100d676d7ff,212.37


In [ ]:
# TODO: get a polars dataframe